In [1]:
from datasets import Dataset
import random
import torch

In [2]:
val_dataset = Dataset.load_from_disk('autodl-tmp/dataset/datasets_val')
val_dataset

Dataset({
    features: ['instruction', 'input', 'response_total', 'response'],
    num_rows: 36869
})

In [3]:
from transformers import AutoTokenizer, AutoModelForCausalLM, DataCollatorForSeq2Seq, TrainingArguments, Trainer, BitsAndBytesConfig

In [4]:
prompt_lst = [
            "你是一个语义分类器。",
            "你将扮演一个文本意图识别器。",
            "你的功能是识别用户输入的语义类别。",
            "你的任务是将文本分配到相应的类别中。",
            "请将自己视为一个文本分类专家。",
            "你的目标是理解并分类文本的意图。",
            "你能够识别和归类不同的文本主题。",
            "你在这里是为了将文本信息分门别类。",
            "你的角色是文本意图的分类器。",
            "你是一个专业的语义识别系统。",
            "你专注于文本的语义分析和分类。",
            "你将提供文本意图的分类服务。",
            "你致力于精确地识别和分类文本内容。",
            "你的任务是分析文本并将其归入适当的类别。",
            "你将帮助识别文本的语义范畴。",
            "你能够快速地对文本进行语义分类。",
            "你的任务是将文本意图进行精确分类。",
            "你是文本语义分类助手。",
            "你将确保文本被正确地分类到其语义类别中。",
            "你的角色是确保文本分类的准确性和效率。",
            "你能为文本提供深入的语义分析和分类。"]  #len = 21

In [5]:
response_lst = [
    "A正面，B负面，C中性，D讽刺，E无产品评价语意",
    "A积极评价，B消极评价，C中立态度，D反讽意味，E无关产品评价的语境",
    "A正面反馈，B负面反馈，C中立观点，D讽刺语气，E非产品评价内容",
    "A正面情绪，B负面情绪，C无明显倾向，D讽刺性表达，E非评价性语义",
    "A赞赏性，B批评性，C客观性，D嘲讽性，E非评价性语义",
    "A正面评价倾向，B负面评价倾向，C评价中立，D讽刺性语气，E非评价性语境",
    "A积极情绪表达，B消极情绪表达，C情绪中性，D讽刺性语调，E非产品评价语义",
    "A正面情感色彩，B负面情感色彩，C情感中立，D讽刺性语气，E非评价性语义",
    "A正面评价，B负面评价，C评价中立，D讽刺性表达，E非产品评价语义",
    "A积极评价倾向，B消极评价倾向，C评价中性，D讽刺性语气，E非评价性语境",
    "A正面情感，B负面情感，C情感中立，D讽刺性语调，E非评价性语义",
    "A正面评价语境，B负面评价语境，C中立评价语境，D讽刺性语境，E非评价性语境",
    "A积极情绪倾向，B消极情绪倾向，C情绪中立，D讽刺性语气，E非产品评价语境",
    "A正面评价性质，B负面评价性质，C评价性质中立，D讽刺性质，E非评价性质",
    "A正面评价特征，B负面评价特征，C评价特征中立，D讽刺特征，E非评价特征",
    "A积极评价语义，B消极评价语义，C评价语义中立，D讽刺语义，E非评价语义",
    "A正面情感倾向性，B负面情感倾向性，C情感倾向性中立，D讽刺倾向性，E非评价倾向性",
    "A正面评价语义特征，B负面评价语义特征，C评价语义中立，D讽刺语义特征，E非评价语义特征",
    "A正面评价语境特征，B负面评价语境特征，C评价语境中立，D讽刺语境特征，E非评价语境特征",
    "A积极评价语义倾向，B消极评价语义倾向，C评价语义中立，D讽刺语义倾向，E非评价语义倾向",
    "A正面情感语义，B负面情感语义，C情感语义中立，D讽刺情感语义，E非情感评价语义"]
def reflash(x,label):
    start = x.find(label)
    end = x.find('，',start)
    return x[start:end]

In [6]:
def process_func_val(example):
    MAX_LENGTH = 386
    input_ids, attention_mask, labels = [], [], []
    instruction_seed = random.randint(0,len(prompt_lst)-1)
    select_seed = random.randint(10,20)
    instruct = prompt_lst[instruction_seed]
    select = response_lst[select_seed]
    get_select = example['response'][0]
    response_select = reflash(select, get_select)
    # 随机替换分类表现选项
    instruction = instruct + (example['input']).replace("\'\'\'","").replace('简要说明理由','不需要说明理由').replace(', "理由": " "','?')
    instruction = instruction.replace("A正面，B负面，C中性，D讽刺，E无产品评价语意",select)
    # instruction = tokenizer(instruction, add_special_tokens = False)
    
    # response = tokenizer(f'<|im_start|>assistant\n"分类标签":"{response_select}"<|im_end|>\n' , add_special_tokens = False)
    # input_ids = instruction['input_ids'] + response['input_ids'] + [tokenizer.eos_token_id]
    # attention_mask = instruction['attention_mask'] + response['attention_mask'] + [1]
    # labels = [-100] * len(instruction['input_ids']) + response['input_ids'] + [tokenizer.eos_token_id]
    # if len(input_ids) > MAX_LENGTH:
    #     input_ids = input_ids[:MAX_LENGTH]
    #     attention_mask = attention_mask[:MAX_LENGTH]
    #     labels = labels[:MAX_LENGTH]
    
    return {'prompt': instruction,  'answer': response_select}

In [7]:
tokenized_val = val_dataset.map(process_func_val, remove_columns=val_dataset.column_names)
tokenized_val

Loading cached processed dataset at /root/autodl-tmp/dataset/datasets_val/cache-6b442126bfe8b7cb.arrow


Dataset({
    features: ['prompt', 'answer'],
    num_rows: 36869
})

In [8]:
tokenized_val[0]

{'prompt': '你是一个语义分类器。#任务：以下是对卷烟产品的评论，按照分类标签内容语义进行评价，不需要说明理由# 评论：review = "太难抽，又苦又涩"# 输出格式： "分类标签": "A正面情感语义，B负面情感语义，C情感语义中立，D讽刺情感语义，E非情感评价语义"?',
 'answer': 'B负面情感语义'}

In [9]:
tokenizer = AutoTokenizer.from_pretrained('autodl-tmp/dpo_qwen_plus_0710', trust_remote_code=True, padding_side='left')
# tokenizer.pad_token = tokenizer.eos_token
# if tokenizer.bos_token is None:   # qwen没有bos_token，要设置一下，不然dpo train时会报错。
#     tokenizer.add_special_tokens({"bos_token": tokenizer.eos_token})
#     tokenizer.bos_token_id = tokenizer.eos_token_id

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [10]:
nf4_config = BitsAndBytesConfig(
   load_in_4bit=True,
   bnb_4bit_quant_type="nf4",
   bnb_4bit_use_double_quant=True,
   bnb_4bit_compute_dtype=torch.bfloat16
)

In [11]:
model = AutoModelForCausalLM.from_pretrained(
    'autodl-tmp/merge_model',
    device_map='auto',
    torch_dtype=torch.bfloat16,
    # cache_dir=cache_dir,
    # quantization_config=nf4_config,
    # low_cpu_mem_usage = True
)
#model

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [12]:
from peft import PeftModel
peft_model = PeftModel.from_pretrained(model, model_id='autodl-tmp/dpo_qwen_plus_0710',torch_type=torch.bfloat16)
#peft_model

In [13]:
peft_model.eval()

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): Qwen2ForCausalLM(
      (model): Qwen2Model(
        (embed_tokens): Embedding(152064, 3584)
        (layers): ModuleList(
          (0-27): 28 x Qwen2DecoderLayer(
            (self_attn): Qwen2SdpaAttention(
              (q_proj): lora.Linear(
                (base_layer): Linear(in_features=3584, out_features=3584, bias=True)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.1, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=3584, out_features=16, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=16, out_features=3584, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
              )
              (k_proj): lora.Linear(
                (base_layer): Linear(in_features=3

In [14]:
tokenized_val[0]

{'prompt': '你是一个语义分类器。#任务：以下是对卷烟产品的评论，按照分类标签内容语义进行评价，不需要说明理由# 评论：review = "太难抽，又苦又涩"# 输出格式： "分类标签": "A正面情感语义，B负面情感语义，C情感语义中立，D讽刺情感语义，E非情感评价语义"?',
 'answer': 'B负面情感语义'}

In [15]:
batchs = []
for i in range(len(tokenized_val)):
    s = tokenized_val[i]['prompt']
    # print(s)
    messages = [
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": s}
    ]
    text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True,
    
    )
    batchs.append(text)

In [16]:
from tqdm import tqdm

In [17]:
responses = []
for i in tqdm(range(0,len(tokenized_val),8)):
    # use batch
    start = i
    end = i + 8
    if end >= len(batchs):
        end = len(batchs)
    s = batchs[start:end]
    
    model_inputs = tokenizer(s, return_tensors="pt",padding=True).to('cuda')
    generated_ids = peft_model.generate(
        model_inputs.input_ids,
        # max_new_tokens=512,
        max_new_tokens=64,
        temperature=0.3,
        pad_token_id=tokenizer.eos_token_id,
        do_sample = True
        
    )
    generated_ids = [
        output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
    ]
    response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)
    # print(response)
    responses.extend(response)

100%|██████████| 4609/4609 [1:04:48<00:00,  1.19it/s]


In [29]:
responses

['"分类标签":"B负面评价语境特征","理由": "评论中的\'太难抽，又苦又涩\'表达了明显的负面感受，直接指出了卷烟产品的口感不佳，体验差，因此属于负面评价。""',
 '"分类标签":"B消极评价语义倾向","理由": "评论中明确表达了产品质量的负面评价，用词如“味道真心不好”和“性价比真的低”直接指出了产品在味道和价格上的不足，并通过与另一产品“陈皮薄荷”的比较，进一步强调了其',
 '"分类标签":"B负面评价语义特征","理由": "评论中的\'没咖啡味，不好抽\'直接表达了用户对卷烟产品的不满，指出产品缺乏期望的味道，并明确表示不喜欢，因此这是一个明显的负面评价。""',
 '"分类标签":"B负面评价性质","理由": "评论中明确表达了对产品的不满，提到了\'毫无咖啡味\'、\'又苦又辣\'以及与低价烟的比较，显示出消费者对产品体验不佳，认为其品质较差，因此这是一个负面评价。""',
 '"分类标签":"B负面评价语境特征","理由": "评论中的“太TM难抽了”表达了用户对卷烟产品的强烈不满，使用了负面的情感词汇“难抽”，直接表达了不好的吸烟体验。""',
 '"分类标签":"B负面情感倾向性","理由": "评论中提到的\'味道不行\'和\'咖啡味淡\'表达了用户对卷烟产品味道的不满，认为其不符合预期，属于负面评价。同时，\'性价比抵\'通常意味着性价比不高，进一步强化了负面评价。""',
 '"分类标签":"B负面评价语境","理由": "评论者认为这款香烟在其价格区间内性价比低，表达了不太满意的情绪，认为如果价格降低到60-70元，则会更值得推荐，这表明当前的价格水平下，产品没有达到用户的预期。""',
 '"分类标签":"B负面评价语境","理由":评论中提到了卷烟的味道“非常的醇香”，入口“很顺”，这些属于正面的评价；同时，评论者也提到了价格“高得有些离谱”，这是一个明显的负面评价。因此，该评论包含了正面和负面的内容',
 '"分类标签":"D讽刺性质","理由": 评论中使用了反问和讽刺的手法，质疑了产品的真实性，暗示产品可能是假冒伪劣的“赝品”，并以此来质疑购买行为的合理性，带有明显的讽刺意味。""',
 '"分类标签":"D讽刺特征","理由": 评论中使用了“难道”、“是不是”，表达了质疑和不满的语气，暗示了品牌产品可能存

In [30]:
dct = {'fine_turing':responses}